In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
from queue import Queue
import json

import requests
from bs4 import BeautifulSoup

In [54]:
def obtain_links(user_input_date):
    user_input_date = str(user_input_date)
    headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
    url = f'https://new.baguio.gov.ph/api/news/search?page=1&office=0&keyword=&from={user_input_date}&to={user_input_date}&admin=undefined&barangay=undefined&public=true'

    response = requests.get(url)
    links = []

    if response.status_code == 200:
        data = json.loads(response.text)
        for item in data["data"]:
            links.append(f'https://new.baguio.gov.ph/news/{item["slug"]}')
    else:
        print("Failed to fetch data")

    if links:
        return links
    else:
        return ['none']

In [61]:
links = obtain_links("2023-04-21")

In [62]:
links

['none']